In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generate synthetic data
def generate_data(num_samples=1000):
    # Generate random input samples of dimension 2
    X = torch.randn(num_samples, 2)
    # Generate targets using a simple linear function + some noise for each dimension
    y = 2 * X[:, 0] + 3 * X[:, 1] + 4 + 0.1 * torch.randn(num_samples)
    return X, y.view(-1, 1)

X_train, y_train = generate_data()

# Define a simple feed-forward neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc = nn.Linear(2, 1)  # Adjusted input dimension to 2

    def forward(self, x):
        return self.fc(x)

# Initialize model, loss, and optimizer
model = SimpleNN()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Train the model
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Export the trained model to ONNX format
# torch.onnx.export(model,               # model being run
#                   X_train,             # model input (or a tuple for multiple inputs)
#                   "simple_model.onnx", # where to save the model (can be a file or file-like object)
#                   export_params=True)  # store the trained parameter weights inside the model file

dynamic_axes = {'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}
torch.onnx.export(model, 
                  X_train,
                  "simple_model.onnx",
                  export_params=True,
                  input_names=['input'],
                  output_names=['output'],
                  dynamic_axes=dynamic_axes)

Epoch [10/100], Loss: 13.3877
Epoch [20/100], Loss: 9.0276
Epoch [30/100], Loss: 6.0943
Epoch [40/100], Loss: 4.1190
Epoch [50/100], Loss: 2.7876
Epoch [60/100], Loss: 1.8893
Epoch [70/100], Loss: 1.2828
Epoch [80/100], Loss: 0.8729
Epoch [90/100], Loss: 0.5955
Epoch [100/100], Loss: 0.4078


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np  # We'll use numpy to generate integer data

# Generate synthetic data
# Generate synthetic data
def generate_data(num_samples=1000):
    # Generate random integer input samples for seq_input and userinfo in a smaller range, say -1000 to 1000
    seq_input = torch.tensor(np.random.randint(-100, 100, size=(num_samples, 16)), dtype=torch.int32)
    userinfo = torch.tensor(np.random.randint(-100, 100, size=(num_samples, 2)), dtype=torch.int32)
    
    # Generate targets as random classes from 0 to 4
    y = torch.randint(0, 5, (num_samples,)).long()
    return seq_input, userinfo, y


seq_input_train, userinfo_train, y_train = generate_data()

# Define a simple feed-forward neural network with dual input
class DualInputNN(nn.Module):
    def __init__(self):
        super(DualInputNN, self).__init__()
        self.fc1 = nn.Linear(16, 8)  # seq_input
        self.fc2 = nn.Linear(2, 2)   # userinfo
        self.fc3 = nn.Linear(10, 5)  # combine seq_input and userinfo

    def forward(self, seq_input, userinfo):
        # Convert the integer inputs to float for processing in the model
        x1 = self.fc1(seq_input.float())
        x2 = self.fc2(userinfo.float())
        x = torch.cat((x1, x2), dim=1)
        return self.fc3(x)

# Initialize model, loss, and optimizer
model = DualInputNN()
criterion = nn.CrossEntropyLoss()  # Modified loss for classification
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
epochs = 100
for epoch in range(epochs):
    # Forward pass
    outputs = model(seq_input_train, userinfo_train)
    loss = criterion(outputs, y_train)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Export the trained model to ONNX format
dynamic_axes = {
    'seq_input': {0: 'batch_size'},
    'userinfo': {0: 'batch_size'},
    'output': {0: 'batch_size'}
}
torch.onnx.export(
    model, 
    (seq_input_train, userinfo_train),
    "dual_input_model.onnx",
    export_params=True,
    input_names=['seq_input', 'userinfo'],
    output_names=['output'],
    dynamic_axes=dynamic_axes
)


Epoch [10/100], Loss: 13.7836
Epoch [20/100], Loss: 8.4760
Epoch [30/100], Loss: 5.1031
Epoch [40/100], Loss: 3.0261
Epoch [50/100], Loss: 2.0223
Epoch [60/100], Loss: 1.7083
Epoch [70/100], Loss: 1.6272
Epoch [80/100], Loss: 1.6046
Epoch [90/100], Loss: 1.5974
Epoch [100/100], Loss: 1.5950


In [9]:
seq_input = numpy.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3]).reshape((1,16))
model(seq_input_train, userinfo_train)

torch.Size([1000, 2])

In [12]:
seq_input = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 3]).reshape((1,16))
userinfo = np.array([1, 1]).reshape((1,2))

In [14]:
model(torch.tensor(seq_input), torch.tensor(userinfo))

tensor([[0.2665, 0.0793, 0.2042, 0.1507, 0.0810]], grad_fn=<AddmmBackward0>)